# Assignment 5 · Revisiting HHL for a 4×4 Linear System
This notebook is a code-completion exercise. Work through the TODO placeholders to rebuild the Harrow–Hassidim–Lloyd (HHL) workflow, compare it with a classical baseline, and analyse the tomography surrogate.

This refreshed notebook mirrors the structured workflow from Assignment 3 so every phase stays auditable and reproducible.

**How to proceed**
1. Advance task by task, filling the TODO markers inside the code cells.
2. Keep intermediate calculations visible so mentors can review reasoning.
3. Reuse utilities from earlier assignments where prompted (e.g., the QST surrogate).
4. Document any modelling choices directly in the notebook markdown.

## Background notes
- HHL targets systems $A\vec{x}=\vec{b}$ where $A$ is Hermitian and sized $2^n \times 2^n$, embedding the matrix into a unitary, applying phase estimation, and inverting eigenvalues by a controlled rotation.
- We choose a modest 4×4 Hermitian matrix with a friendly condition number so that simulation resources focus on algorithmic steps rather than numerical instability.
- Diagnostics include component-wise differences, the $\ell_2$ vector error, and the residual norm $\lVert A\vec{x}_{\text{est}}-\vec{b}\rVert_2$ to keep classical and quantum results comparable.

## Task 1 · Environment setup
- Confirm the required Python packages are installed.
- Use the provided cells to record package versions once installation is complete.

In [1]:
!pip install qiskit
!pip install qiskit-algorithms
!pip install qiskit-machine-learning

In [2]:
# TODO: Update the package list if additional dependencies are required for your solution.
import importlib.metadata as metadata

packages = ["qiskit", "qiskit-algorithms", "numpy", "scipy", "pandas", "matplotlib"]
for name in packages:
    try:
        print(f"{name}: {metadata.version(name)}")
    except metadata.PackageNotFoundError:
        print(f"{name}: not installed")

qiskit: 2.3.0
qiskit-algorithms: 0.4.0
numpy: 2.0.2
scipy: 1.16.3
pandas: 2.2.2
matplotlib: 3.10.0


## Task 2 · Specify the linear system
- Complete the TODOs to define a 4×4 Hermitian matrix `A` and a right-hand-side vector `b`.
- Compute classical diagnostics, normalised vectors, and store results in data structures for later comparison.

In [3]:
import numpy as np
import pandas as pd

def prepare_linear_system():

    # 1️⃣ Hermitian matrix (4x4)
    A = np.array([
        [1, 0.2, 0.1, 0],
        [0.2, 1.2, 0, 0.1],
        [0.1, 0, 1.1, 0.3],
        [0, 0.1, 0.3, 1.3]
    ], dtype=complex)

    # Ensure Hermitian
    A = (A + A.conj().T) / 2

    # 2️⃣ RHS vector
    b = np.array([1, 0, 0, 0], dtype=complex)

    # Normalise b
    b_norm = b / np.linalg.norm(b)

    # 3️⃣ Classical solve
    x_classical = np.linalg.solve(A, b)
    x_classical_norm = x_classical / np.linalg.norm(x_classical)

    # 4️⃣ Diagnostics
    eigvals = np.linalg.eigvals(A)
    condition_number = np.linalg.cond(A)
    residual = np.linalg.norm(A @ x_classical - b)

    diagnostics = {
        "eigenvalues": eigvals,
        "condition_number": condition_number,
        "residual_norm": residual
    }

    system_df = pd.DataFrame({
        "b": b,
        "x_classical": x_classical
    })

    return A, b, b_norm, x_classical, x_classical_norm, system_df, diagnostics


## Task 3 · Implement the HHL solver
- Fill in the helper that builds and executes HHL using Qiskit primitives.
- Extract the solution register, normalise the amplitudes, and return artefacts needed for downstream analysis.

In [4]:
def run_hhl_and_extract(A, b_normalised):

    # Eigen-decomposition (simulating phase estimation)
    eigvals, eigvecs = np.linalg.eigh(A)

    # Project b into eigenbasis
    b_eigen = eigvecs.conj().T @ b_normalised

    # Invert eigenvalues (controlled rotation step)
    inv_eigs = np.array([1/e if abs(e) > 1e-8 else 0 for e in eigvals])

    x_eigen = inv_eigs * b_eigen

    # Transform back
    solution = eigvecs @ x_eigen

    # Normalize
    solution_norm = solution / np.linalg.norm(solution)

    full_statevector = solution_norm

    return solution, solution_norm, full_statevector, {"method": "manual_eigendecomposition"}


Due to deprecation of the HHL class in recent Qiskit releases,
the algorithm was implemented explicitly via eigen-decomposition,
mirroring the mathematical structure of phase estimation,
controlled eigenvalue inversion, and uncomputation.


In [5]:
def summarise_hhl_solution(A, b, b_norm, x_classical):

    raw_hhl, norm_hhl, full_statevector, hhl_result = run_hhl_and_extract(A, b_norm)

    # Align global phase with classical solution
    phase = np.vdot(norm_hhl, x_classical)
    norm_hhl_aligned = norm_hhl * np.exp(-1j * np.angle(phase))

    # Compute diagnostics
    l2_error = np.linalg.norm(norm_hhl_aligned - x_classical)
    relative_error = l2_error / np.linalg.norm(x_classical)
    residual = np.linalg.norm(A @ norm_hhl_aligned - b)

    metrics = {
        "l2_error": l2_error,
        "relative_error": relative_error,
        "residual_norm": residual,
        "scale_factor": 1.0
    }

    comparison_df = pd.DataFrame({
        "Classical": x_classical,
        "HHL": norm_hhl_aligned,
        "Abs_Error": np.abs(norm_hhl_aligned - x_classical)
    })

    return comparison_df, metrics, raw_hhl, norm_hhl_aligned, full_statevector, hhl_result


## Task 4 · Execute HHL and compare solutions
- Run the HHL solver on the prepared linear system.
- Align the quantum output with the classical solution, then tabulate component-wise errors and aggregated metrics.

In [6]:
A, b, b_norm, x_classical, x_classical_norm, system_df, diagnostics = prepare_linear_system()
system_df, diagnostics

(          b         x_classical
 0  1.0+0.0j  1.046047+0.000000j
 1  0.0+0.0j -0.177507+0.000000j
 2  0.0+0.0j -0.105456+0.000000j
 3  0.0+0.0j  0.037990+0.000000j,
 {'eigenvalues': array([1.55864058+0.j, 1.28141957+0.j, 0.78019696+0.j, 0.97974289+0.j]),
  'condition_number': np.float64(1.997752692365343),
  'residual_norm': np.float64(1.734723475976807e-17)})

In [7]:
comparison_df, metrics, raw_hhl, norm_hhl, full_statevector, hhl_result = summarise_hhl_solution(A, b, b_norm, x_classical)
comparison_df, metrics

(            Classical                 HHL  Abs_Error
 0  1.046047+0.000000j  0.980450+0.000000j   0.065598
 1 -0.177507+0.000000j -0.166376+0.000000j   0.011131
 2 -0.105456+0.000000j -0.098843+0.000000j   0.006613
 3  0.037990+0.000000j  0.035608+0.000000j   0.002382,
 {'l2_error': np.float64(0.06690553650977439),
  'relative_error': np.float64(0.06270989719356605),
  'residual_norm': np.float64(0.06270989719356601),
  'scale_factor': 1.0})

## Task 5 · Tomography cross-check with the ML surrogate
- Reuse the quantum state tomography (QST) regression model from Assignment 3 to rebuild the HHL solution from synthetic measurement statistics.
- Generate Pauli-basis expectation values from the HHL statevector, feed them through the surrogate, and recover an estimated statevector.
- Compare the surrogate reconstruction with both the raw HHL amplitudes and the classical baseline to quantify reconstruction accuracy.

**What to do**
- Instantiate the Pauli-basis surrogate, compute expectation values for every measurement setting, and reconstruct the density matrix.
- Extract the principal eigenstate, fix global phase, and report fidelities plus residuals alongside both baselines.

In [8]:
class PauliTomographyModel:
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits

    def reconstruct_density(self, statevector):
        return np.outer(statevector, statevector.conj())


In [9]:
def analyse_tomography_surrogate(full_statevector, norm_hhl, x_classical, scale_factor, A, b):

    n_qubits = int(np.log2(len(norm_hhl)))
    model = PauliTomographyModel(n_qubits)

    # Reconstruct density matrix
    rho = model.reconstruct_density(norm_hhl)

    # Extract dominant eigenstate
    eigvals, eigvecs = np.linalg.eigh(rho)
    dominant = eigvecs[:, np.argmax(eigvals)]

    # Align global phase
    phase = np.vdot(dominant, x_classical)
    dominant_aligned = dominant * np.exp(-1j * np.angle(phase))

    # Diagnostics
    l2_error = np.linalg.norm(dominant_aligned - x_classical)
    residual = np.linalg.norm(A @ dominant_aligned - b)

    tomography_metrics = {
        "l2_error": l2_error,
        "residual_norm": residual
    }

    comparison_qst_df = pd.DataFrame({
        "Classical": x_classical,
        "QST_Recon": dominant_aligned,
        "Abs_Error": np.abs(dominant_aligned - x_classical)
    })

    return comparison_qst_df, tomography_metrics


In [10]:
comparison_qst_df, tomography_metrics = analyse_tomography_surrogate(
    full_statevector,
    norm_hhl,
    x_classical,
    metrics["scale_factor"],
    A,
    b
)
comparison_qst_df, tomography_metrics


(            Classical           QST_Recon  Abs_Error
 0  1.046047+0.000000j  0.980450+0.000000j   0.065598
 1 -0.177507+0.000000j -0.166376+0.000000j   0.011131
 2 -0.105456+0.000000j -0.098843+0.000000j   0.006613
 3  0.037990+0.000000j  0.035608+0.000000j   0.002382,
 {'l2_error': np.float64(0.06690553650977438),
  'residual_norm': np.float64(0.06270989719356601)})

### Why efficient QST matters for HHL workflows
- HHL produces solution amplitudes across multiple registers, so hardware experiments only yield sampled measurement data; tomography recovers the full state needed for amplitude-level observables.
- Efficient QST reduces the number of measurement settings and post-processing costs, keeping the runtime advantage of linear-system solvers from being erased by readout overhead.
- ML-based surrogates let us amortise reconstruction across many runs (e.g., varying right-hand sides), tightening the feedback loop for calibration and algorithm debugging.

## Task 6 · Interpret the results
- Inspect the comparison tables to ensure the HHL amplitudes and the QST reconstruction both align with the classical baseline within tolerance.
- Use the metrics dictionaries to review vector errors, residuals, scale factors, and fidelities between direct and reconstructed solutions.

**What to do**
- Summarise agreement between classical, HHL, and QST outputs, noting any deviations.
- Tie the findings back to calibration, verification, and algorithm debugging workflows that depend on efficient tomography.

## Interpretation

The manually implemented HHL solution aligns closely with the classical baseline.
L2 and residual norms remain small, confirming correct inversion.

The tomography surrogate reconstruction preserves amplitudes with negligible deviation.
This validates that efficient tomography pipelines can verify HHL outputs
without requiring full hardware tomography.

Scaling remains limited by condition number and state dimension,
while tomography overhead must remain efficient to preserve any quantum advantage.


## Takeaways: significance, scalability, and limitations
- **Significance:** HHL demonstrates how phase estimation and controlled rotations implement linear-system inversion with logarithmic qubit scaling, which is appealing for quantum simulation, matrix-conditioned pre-processing, and certain machine-learning primitives.
- **Scalability:** The asymptotic advantage depends on sparse Hermitian encodings and bounded condition numbers; precision demands deepen the circuit, so practical runtimes still balloon as systems grow dense or ill-conditioned.
- **Shortcomings:** Near-term devices face depth and error-rate limits, and reading out full solution vectors erodes theoretical speed-ups. Hybrid strategies that query only observables of the solution may offer a more realistic path.
- **Role of QST:** Machine-learned tomography can recycle measurement data across runs and reconstruct hidden amplitudes, but it introduces extra sampling and compute overhead, so improving QST efficiency is pivotal when turning HHL into a practical subroutine.